# Excel 分割器 - SharePoint Python 整合版 v2.0

<!-- 
開發日期: 2025年1月
功能: 完全使用 Python 處理 SharePoint 分享，支援 SSO 認證
v2.0: 修復資料驗證問題
-->

這個筆記本提供了一個完整的 Python 解決方案，用於分割 Excel 檔案並透過 SharePoint API 自動分享。

## 主要特色
- 🔐 使用 SSO (單一登入) 進行認證
- 📂 自動建立審查者專屬資料夾
- 📊 按審查者分割 Excel 檔案
- ✅ **保留資料驗證規則**（v2.0 新增）
- 📧 自動查找使用者 email 地址
- ✅ 互動式審查者選擇介面
- 📤 批次 SharePoint 分享功能
- 📊 即時進度顯示

## 系統需求
- Python 3.9 或更高版本
- Microsoft 365 帳號與 SharePoint 存取權限
- 已設定 SSO 登入的電腦

## v2.0 更新內容
- 修復資料驗證遺失問題
- 新增進階處理選項
- 改進檔案處理邏輯

## 步驟 1: 安裝必要套件

In [ ]:
# 安裝必要套件
import sys
import subprocess

required_packages = [
    'pandas',
    'openpyxl',
    'ipywidgets',
    'msal',          # Microsoft 認證函式庫
    'requests',      # HTTP 請求
    'python-dotenv', # 環境變數管理
    'xlsxwriter'     # 進階 Excel 處理 (v2.0 新增)
]

print("🔍 檢查套件安裝狀態...")
for package in required_packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✓ {package} 已安裝")
    except ImportError:
        print(f"📦 正在安裝 {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} 安裝完成")

print("\n✅ 所有必要套件已就緒！")

## 步驟 2: 匯入函式庫與設定

In [ ]:
import os
import shutil
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import glob
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import json
import requests
import msal
import webbrowser
from urllib.parse import quote
import time
from typing import Dict, List, Optional, Tuple
import re
from dotenv import load_dotenv
import xlsxwriter

# 載入環境變數
load_dotenv()

# 檢查 tkinter (檔案對話框)
try:
    import tkinter as tk
    from tkinter import filedialog
    TKINTER_AVAILABLE = True
except ImportError:
    TKINTER_AVAILABLE = False

# 檢查 win32com (Windows Excel 自動化)
try:
    import win32com.client
    WIN32COM_AVAILABLE = True
except ImportError:
    WIN32COM_AVAILABLE = False

print("✓ 函式庫匯入成功")
print(f"✓ 檔案對話框: {'可用' if TKINTER_AVAILABLE else '不可用'}")
print(f"✓ Excel 自動化: {'可用' if WIN32COM_AVAILABLE else '不可用'}")

# Microsoft Graph API 設定
GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0'

# 所需的權限範圍
SCOPES = [
    'User.Read',
    'User.ReadBasic.All',
    'Sites.ReadWrite.All',
    'Files.ReadWrite.All'
]

# 全域變數
access_token = None
msal_app = None
current_user = None

## 步驟 3: SSO 認證功能

In [ ]:
def get_auth_config():
    """取得認證設定"""
    # 優先從環境變數讀取
    client_id = os.getenv('AZURE_CLIENT_ID', '')
    tenant_id = os.getenv('AZURE_TENANT_ID', '')
    
    # 如果沒有環境變數，使用預設值（稍後從 UI 輸入）
    return {
        'client_id': client_id,
        'tenant_id': tenant_id,
        'redirect_uri': 'http://localhost:8400'
    }

def initialize_msal_app(client_id=None, tenant_id=None):
    """初始化 MSAL 應用程式"""
    global msal_app
    
    config = get_auth_config()
    
    # 使用提供的值覆蓋設定
    if client_id:
        config['client_id'] = client_id
    if tenant_id:
        config['tenant_id'] = tenant_id
    
    if not config['client_id'] or not config['tenant_id']:
        return False
    
    authority = f"https://login.microsoftonline.com/{config['tenant_id']}"
    
    # 建立 MSAL 應用程式，啟用 SSO
    msal_app = msal.PublicClientApplication(
        config['client_id'],
        authority=authority,
        # 啟用 token 快取
        token_cache=msal.SerializableTokenCache()
    )
    
    return True

def authenticate_sso():
    """使用 SSO 進行認證"""
    global access_token, current_user
    
    if not msal_app:
        print("❌ MSAL 應用程式未初始化")
        return None
    
    # 首先嘗試從快取取得 token
    accounts = msal_app.get_accounts()
    
    if accounts:
        # 找到帳號，嘗試靜默取得 token
        print(f"🔍 找到快取的帳號: {accounts[0]['username']}")
        
        result = msal_app.acquire_token_silent(
            scopes=SCOPES,
            account=accounts[0]
        )
        
        if result and 'access_token' in result:
            access_token = result['access_token']
            current_user = accounts[0]['username']
            print(f"✅ 使用快取登入成功: {current_user}")
            return access_token
    
    # 如果靜默認證失敗，使用互動式登入
    print("🌐 開啟瀏覽器進行登入...")
    
    try:
        # 使用裝置碼流程（適合 Jupyter）
        flow = msal_app.initiate_device_flow(scopes=SCOPES)
        
        if 'user_code' not in flow:
            print("❌ 無法啟動裝置碼流程")
            return None
        
        print("\n" + "="*50)
        print(f"👉 請前往: {flow['verification_uri']}")
        print(f"👉 輸入代碼: {flow['user_code']}")
        print("="*50 + "\n")
        
        # 自動開啟瀏覽器
        webbrowser.open(flow['verification_uri'])
        
        # 等待使用者完成認證
        result = msal_app.acquire_token_by_device_flow(flow)
        
        if 'access_token' in result:
            access_token = result['access_token']
            current_user = result.get('account', {}).get('username', '未知')
            print(f"\n✅ 登入成功: {current_user}")
            
            # 儲存 token 快取
            if msal_app.token_cache.has_state_changed:
                # 可以將快取儲存到檔案供下次使用
                pass
            
            return access_token
        else:
            error = result.get('error_description', '未知錯誤')
            print(f"\n❌ 登入失敗: {error}")
            return None
            
    except Exception as e:
        print(f"\n❌ 認證過程發生錯誤: {str(e)}")
        return None

def check_authentication():
    """檢查認證狀態"""
    if access_token:
        # 測試 token 是否有效
        headers = {'Authorization': f'Bearer {access_token}'}
        response = requests.get(f"{GRAPH_API_ENDPOINT}/me", headers=headers)
        
        if response.status_code == 200:
            user_info = response.json()
            return True, user_info.get('displayName', current_user)
        else:
            return False, None
    return False, None

print("✓ SSO 認證功能已就緒")

## 步驟 4: API 呼叫與錯誤處理

In [ ]:
def make_api_call(endpoint: str, method: str = 'GET', data: dict = None, 
                 headers: dict = None, retry_count: int = 3) -> dict:
    """執行 API 呼叫與自動重試"""
    if not access_token:
        print("❌ 尚未認證，請先登入")
        return None
    
    default_headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    
    if headers:
        default_headers.update(headers)
    
    for attempt in range(retry_count):
        try:
            if method == 'GET':
                response = requests.get(endpoint, headers=default_headers, timeout=30)
            elif method == 'POST':
                response = requests.post(endpoint, headers=default_headers, json=data, timeout=30)
            elif method == 'PATCH':
                response = requests.patch(endpoint, headers=default_headers, json=data, timeout=30)
            elif method == 'PUT':
                response = requests.put(endpoint, headers=default_headers, json=data, timeout=30)
            else:
                raise ValueError(f"不支援的方法: {method}")
            
            # 成功回應
            if response.status_code in [200, 201, 204]:
                return response.json() if response.content else {}
            
            # 速率限制
            elif response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 2 ** attempt))
                print(f"⏳ 達到速率限制，等待 {retry_after} 秒...")
                time.sleep(retry_after)
                continue
            
            # 認證過期
            elif response.status_code == 401:
                print("🔄 認證已過期，嘗試重新登入...")
                if authenticate_sso():
                    default_headers['Authorization'] = f'Bearer {access_token}'
                    continue
                else:
                    return None
            
            # 其他錯誤
            else:
                error_msg = f"API 錯誤 {response.status_code}"
                try:
                    error_detail = response.json()
                    error_msg += f": {error_detail.get('error', {}).get('message', response.text[:200])}"
                except:
                    error_msg += f": {response.text[:200]}"
                
                print(f"❌ {error_msg}")
                
                if attempt < retry_count - 1:
                    wait_time = 2 ** attempt
                    print(f"⏳ {wait_time} 秒後重試...")
                    time.sleep(wait_time)
                else:
                    return None
                    
        except requests.exceptions.Timeout:
            print(f"⏱️ 請求逾時 (嘗試 {attempt + 1}/{retry_count})")
            if attempt < retry_count - 1:
                time.sleep(2 ** attempt)
            else:
                return None
                
        except requests.exceptions.ConnectionError:
            print(f"🔌 連線錯誤 (嘗試 {attempt + 1}/{retry_count})")
            if attempt < retry_count - 1:
                time.sleep(2 ** attempt)
            else:
                return None
                
        except Exception as e:
            print(f"❌ 發生錯誤: {str(e)}")
            return None
    
    return None

print("✓ API 呼叫函數已就緒")

## 步驟 5: SharePoint 整合功能

In [ ]:
def sanitize_folder_name(name: str) -> str:
    """清理資料夾名稱，確保相容性"""
    # 移除或替換不允許的字元
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '#', '%']
    sanitized = name.strip()
    
    for char in invalid_chars:
        sanitized = sanitized.replace(char, '_')
    
    # 限制長度
    if len(sanitized) > 255:
        sanitized = sanitized[:255].rstrip()
    
    return sanitized

def validate_email(email: str) -> bool:
    """驗證 email 格式"""
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def lookup_user_email(display_name: str) -> Optional[str]:
    """從 Microsoft 365 查找使用者 email"""
    if not access_token:
        return None
    
    try:
        # 處理特殊字元
        escaped_name = display_name.replace("'", "''")
        
        # 搜尋使用者
        search_filter = f"displayName eq '{escaped_name}' or startswith(displayName, '{escaped_name}')"
        endpoint = f"{GRAPH_API_ENDPOINT}/users?$filter={quote(search_filter)}&$select=displayName,mail,userPrincipalName&$top=10"
        
        result = make_api_call(endpoint)
        
        if result and 'value' in result:
            users = result['value']
            
            # 尋找完全符合的
            for user in users:
                if user.get('displayName', '').lower() == display_name.lower():
                    email = user.get('mail') or user.get('userPrincipalName')
                    if email and validate_email(email):
                        return email
            
            # 如果沒有完全符合，返回第一個結果
            if users:
                email = users[0].get('mail') or users[0].get('userPrincipalName')
                if email and validate_email(email):
                    return email
                    
    except Exception as e:
        print(f"查找 {display_name} 時發生錯誤: {str(e)}")
    
    return None

def get_sharepoint_site_info(site_url: str) -> Tuple[Optional[str], Optional[str]]:
    """取得 SharePoint 網站資訊"""
    try:
        # 從 URL 提取網站路徑
        if 'sharepoint.com' in site_url:
            # 提取主機名稱
            import urllib.parse
            parsed = urllib.parse.urlparse(site_url)
            hostname = parsed.hostname
            
            # 提取網站路徑
            path_parts = parsed.path.strip('/').split('/')
            
            if len(path_parts) >= 2 and path_parts[0] == 'sites':
                site_name = path_parts[1]
                
                # 取得網站 ID
                endpoint = f"{GRAPH_API_ENDPOINT}/sites/{hostname}:/sites/{site_name}"
                result = make_api_call(endpoint)
                
                if result and 'id' in result:
                    return result['id'], result.get('webUrl')
        
    except Exception as e:
        print(f"取得網站資訊時發生錯誤: {str(e)}")
    
    return None, None

def get_or_create_folder(site_id: str, folder_path: str) -> Optional[str]:
    """取得或建立 SharePoint 資料夾"""
    try:
        # 檢查資料夾是否存在
        check_endpoint = f"{GRAPH_API_ENDPOINT}/sites/{site_id}/drive/root:/{quote(folder_path, safe='')}"
        result = make_api_call(check_endpoint)
        
        if result and 'id' in result:
            return result['id']
        
        # 如果不存在，建立資料夾
        parent_path = '/'.join(folder_path.split('/')[:-1]) if '/' in folder_path else ''
        folder_name = folder_path.split('/')[-1]
        
        if parent_path:
            create_endpoint = f"{GRAPH_API_ENDPOINT}/sites/{site_id}/drive/root:/{quote(parent_path, safe='')}:/children"
        else:
            create_endpoint = f"{GRAPH_API_ENDPOINT}/sites/{site_id}/drive/root/children"
        
        data = {
            "name": folder_name,
            "folder": {},
            "@microsoft.graph.conflictBehavior": "rename"
        }
        
        result = make_api_call(create_endpoint, method='POST', data=data)
        
        if result and 'id' in result:
            return result['id']
            
    except Exception as e:
        print(f"處理資料夾 {folder_path} 時發生錯誤: {str(e)}")
    
    return None

def share_folder_with_user(site_id: str, folder_path: str, user_email: str, 
                          permission: str = 'write', notify: bool = False) -> bool:
    """分享資料夾給特定使用者"""
    try:
        # 確保資料夾存在
        folder_id = get_or_create_folder(site_id, folder_path)
        if not folder_id:
            print(f"❌ 無法取得資料夾: {folder_path}")
            return False
        
        # 建立分享連結
        endpoint = f"{GRAPH_API_ENDPOINT}/sites/{site_id}/drive/items/{folder_id}/createLink"
        
        data = {
            "type": permission,  # 'view' 或 'write'
            "scope": "users",   # 只分享給特定使用者
            "recipients": [
                {
                    "email": user_email
                }
            ]
        }
        
        if notify:
            data["sendNotification"] = True
            data["message"] = f"您已獲得存取 '{folder_path}' 資料夾的權限。"
        
        result = make_api_call(endpoint, method='POST', data=data)
        
        if result and 'link' in result:
            return True
        else:
            # 嘗試使用 invite 方法
            invite_endpoint = f"{GRAPH_API_ENDPOINT}/sites/{site_id}/drive/items/{folder_id}/invite"
            
            invite_data = {
                "requireSignIn": True,
                "sendInvitation": notify,
                "roles": [permission],
                "recipients": [
                    {
                        "email": user_email
                    }
                ]
            }
            
            if notify:
                invite_data["message"] = f"您已獲得存取 '{folder_path}' 資料夾的權限。"
            
            result = make_api_call(invite_endpoint, method='POST', data=invite_data)
            return result is not None
            
    except Exception as e:
        print(f"分享資料夾時發生錯誤: {str(e)}")
        return False

def upload_file_to_sharepoint(site_id: str, local_file_path: str, 
                             sharepoint_folder: str) -> bool:
    """上傳檔案到 SharePoint"""
    try:
        # 確保資料夾存在
        folder_id = get_or_create_folder(site_id, sharepoint_folder)
        if not folder_id:
            return False
        
        file_name = os.path.basename(local_file_path)
        file_size = os.path.getsize(local_file_path)
        
        # 小檔案直接上傳（< 4MB）
        if file_size < 4 * 1024 * 1024:
            with open(local_file_path, 'rb') as f:
                file_content = f.read()
            
            endpoint = f"{GRAPH_API_ENDPOINT}/sites/{site_id}/drive/items/{folder_id}:/{quote(file_name, safe='')}:/content"
            
            headers = {
                'Authorization': f'Bearer {access_token}',
                'Content-Type': 'application/octet-stream'
            }
            
            response = requests.put(endpoint, headers=headers, data=file_content)
            return response.status_code in [200, 201]
        
        else:
            # 大檔案使用分段上傳
            # 這裡簡化處理，實際應實作分段上傳
            print(f"⚠️ 檔案 {file_name} 太大 ({file_size / 1024 / 1024:.1f}MB)，暫不支援")
            return False
            
    except Exception as e:
        print(f"上傳檔案時發生錯誤: {str(e)}")
        return False

print("✓ SharePoint 整合功能已就緒")

## 步驟 6: 檔案處理功能（v2.0 修復資料驗證）

In [ ]:
def find_column(worksheet, column_name):
    """在工作表中尋找欄位"""
    for col_idx, cell in enumerate(worksheet[1], start=1):
        if cell.value == column_name:
            return col_idx
    raise ValueError(f"找不到 '{column_name}' 欄位！")

def copy_selected_documents(source_dir, dest_dir, copy_word=True, copy_pdf=True):
    """複製選定的文件類型"""
    copied_files = []
    
    if copy_word:
        word_patterns = [
            os.path.join(source_dir, "*.docx"),
            os.path.join(source_dir, "*.doc")
        ]
        
        for pattern in word_patterns:
            for file in glob.glob(pattern):
                if os.path.isfile(file):
                    dest_path = os.path.join(dest_dir, os.path.basename(file))
                    shutil.copy2(file, dest_path)
                    copied_files.append(os.path.basename(file))
    
    if copy_pdf:
        pdf_pattern = os.path.join(source_dir, "*.pdf")
        for file in glob.glob(pdf_pattern):
            if os.path.isfile(file):
                dest_path = os.path.join(dest_dir, os.path.basename(file))
                shutil.copy2(file, dest_path)
                copied_files.append(os.path.basename(file))
    
    return copied_files

def process_reviewer_excel(file_path, reviewer, column_name, output_folder, processing_method='delete_rows'):
    """為特定審查者建立篩選過的 Excel 檔案（v2.0 - 保留資料驗證）"""
    try:
        # 清理審查者名稱
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        
        # 建立審查者資料夾
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        # 建立新檔名
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        ext = os.path.splitext(base_name)[1]
        new_filename = f"{name_without_ext} - {reviewer_name}{ext}"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        if processing_method == 'delete_rows':
            # 方法 1: 刪除列（保留資料驗證）
            # 重要：使用 data_only=False 和 keep_vba=True 來保留所有功能
            wb = load_workbook(file_path, data_only=False, keep_vba=True, keep_links=True)
            
            # 確保複製所有工作表（包含資料驗證來源）
            # 不要只處理 active sheet
            main_ws = wb.active
            
            # 尋找欄位
            col_idx = find_column(main_ws, column_name)
            
            # 收集要保留的列索引
            rows_to_keep = [1]  # 保留標題列
            for row in range(2, main_ws.max_row + 1):
                cell_value = main_ws.cell(row=row, column=col_idx).value
                if str(cell_value) == str(reviewer):
                    rows_to_keep.append(row)
            
            # 反向刪除不需要的列（從底部開始以避免索引問題）
            for row in range(main_ws.max_row, 1, -1):
                if row not in rows_to_keep:
                    main_ws.delete_rows(row)
            
            # 儲存檔案
            wb.save(dst_path)
            wb.close()
            
        elif processing_method == 'copy_file_first':
            # 方法 2: 先複製整個檔案再處理
            # 先複製整個檔案
            shutil.copy2(file_path, dst_path)
            
            # 使用 openpyxl 處理
            wb = load_workbook(dst_path, data_only=False, keep_vba=True, keep_links=True)
            main_ws = wb.active
            
            # 保留資料驗證資訊
            data_validations = main_ws.data_validations.dataValidation
            
            # 尋找並刪除非相關列
            col_idx = find_column(main_ws, column_name)
            rows_to_delete = []
            
            for row in range(2, main_ws.max_row + 1):
                cell_value = main_ws.cell(row=row, column=col_idx).value
                if str(cell_value) != str(reviewer):
                    rows_to_delete.append(row)
            
            # 從底部開始刪除
            for row in sorted(rows_to_delete, reverse=True):
                main_ws.delete_rows(row)
            
            # 重新應用資料驗證（如果需要）
            for dv in data_validations:
                main_ws.add_data_validation(dv)
            
            wb.save(dst_path)
            wb.close()
            
        elif processing_method == 'win32com' and WIN32COM_AVAILABLE:
            # 方法 3: 使用 win32com（最可靠但僅限 Windows）
            excel = win32com.client.Dispatch("Excel.Application")
            excel.Visible = False
            excel.DisplayAlerts = False
            
            try:
                # 開啟原始檔案
                wb = excel.Workbooks.Open(os.path.abspath(file_path))
                ws = wb.ActiveSheet
                
                # 尋找欄位
                header_row = ws.Rows(1)
                col_idx = None
                for col in range(1, ws.UsedRange.Columns.Count + 1):
                    if ws.Cells(1, col).Value == column_name:
                        col_idx = col
                        break
                
                if not col_idx:
                    raise ValueError(f"找不到欄位 '{column_name}'")
                
                # 套用篩選
                ws.AutoFilterMode = False
                ws.UsedRange.AutoFilter(Field=col_idx, Criteria1=str(reviewer))
                
                # 刪除隱藏的列
                for row in range(ws.UsedRange.Rows.Count, 1, -1):
                    if ws.Rows(row).Hidden:
                        ws.Rows(row).Delete()
                
                # 移除篩選
                ws.AutoFilterMode = False
                
                # 另存新檔
                wb.SaveAs(os.path.abspath(dst_path))
                wb.Close()
                
            finally:
                excel.Quit()
        
        else:
            # 預設方法：隱藏列（原始方法，可能會遺失資料驗證）
            wb = load_workbook(file_path)
            ws = wb.active
            
            # 尋找欄位並套用篩選
            col_idx = find_column(ws, column_name)
            max_row = ws.max_row
            max_col = ws.max_column
            
            # 設定自動篩選
            filter_range = f"A1:{get_column_letter(max_col)}{max_row}"
            ws.auto_filter.ref = filter_range
            ws.auto_filter.add_filter_column(col_idx - 1, [str(reviewer)])
            
            # 隱藏不符合的列
            for row in range(2, max_row + 1):
                cell_value = ws.cell(row=row, column=col_idx).value
                if str(cell_value) != str(reviewer):
                    ws.row_dimensions[row].hidden = True
            
            # 儲存檔案
            wb.save(dst_path)
            wb.close()
        
        return True, reviewer_folder, new_filename
        
    except Exception as e:
        print(f"處理 {reviewer} 的檔案時發生錯誤: {str(e)}")
        return False, None, None

print("✓ 檔案處理功能已就緒（v2.0 - 支援保留資料驗證）")

## 步驟 7: 使用者介面

In [ ]:
# 全域變數
last_folder = os.path.expanduser("~")
reviewer_data = {}  # 儲存審查者資料
site_id = None
site_url = None

# === 認證介面 ===
display(HTML("<h3>🔐 步驟 1: 認證設定</h3>"))

# Azure AD 設定
client_id_input = widgets.Text(
    value=os.getenv('AZURE_CLIENT_ID', ''),
    placeholder='輸入 Azure AD Client ID',
    description='Client ID:',
    layout=widgets.Layout(width='400px')
)

tenant_id_input = widgets.Text(
    value=os.getenv('AZURE_TENANT_ID', ''),
    placeholder='輸入 Azure AD Tenant ID',
    description='Tenant ID:',
    layout=widgets.Layout(width='400px')
)

auth_status = widgets.HTML(value="<b>狀態:</b> 尚未認證")
auth_button = widgets.Button(
    description='登入',
    layout=widgets.Layout(width='100px')
)

def on_auth_click(b):
    """處理認證按鈕點擊"""
    with output:
        clear_output(wait=True)
        
        # 初始化 MSAL
        if initialize_msal_app(client_id_input.value.strip(), tenant_id_input.value.strip()):
            # 執行 SSO 認證
            if authenticate_sso():
                auth_status.value = f"<b>狀態:</b> <span style='color:green'>✓ 已登入為 {current_user}</span>"
                auth_button.description = "重新登入"
                
                # 儲存設定到環境變數（選擇性）
                os.environ['AZURE_CLIENT_ID'] = client_id_input.value.strip()
                os.environ['AZURE_TENANT_ID'] = tenant_id_input.value.strip()
            else:
                auth_status.value = "<b>狀態:</b> <span style='color:red'>✗ 登入失敗</span>"
        else:
            print("❌ 請輸入有效的 Client ID 和 Tenant ID")

auth_button.on_click(on_auth_click)

display(client_id_input)
display(tenant_id_input)
display(widgets.HBox([auth_button, auth_status]))

# === SharePoint 設定 ===
display(HTML("<h3>🌐 步驟 2: SharePoint 設定</h3>"))

sharepoint_url_input = widgets.Text(
    value='',
    placeholder='https://company.sharepoint.com/sites/teamsite',
    description='SharePoint 網站:',
    layout=widgets.Layout(width='500px')
)

verify_site_button = widgets.Button(
    description='驗證網站',
    layout=widgets.Layout(width='100px')
)

site_status = widgets.HTML(value="")

def on_verify_site(b):
    """驗證 SharePoint 網站"""
    global site_id, site_url
    with output:
        clear_output(wait=True)
        
        if not access_token:
            print("❌ 請先完成登入")
            return
        
        url = sharepoint_url_input.value.strip()
        if not url:
            print("❌ 請輸入 SharePoint 網站 URL")
            return
        
        print(f"🔍 驗證網站: {url}")
        sid, web_url = get_sharepoint_site_info(url)
        
        if sid:
            site_id = sid
            site_url = web_url
            site_status.value = "<span style='color:green'>✓ 網站已驗證</span>"
            print(f"✅ 網站已驗證")
        else:
            site_status.value = "<span style='color:red'>✗ 無法驗證網站</span>"
            print("❌ 無法驗證網站，請檢查 URL")

verify_site_button.on_click(on_verify_site)

display(sharepoint_url_input)
display(widgets.HBox([verify_site_button, site_status]))

# === 檔案選擇 ===
display(HTML("<h3>📁 步驟 3: 檔案選擇</h3>"))

def select_excel_file():
    """選擇 Excel 檔案"""
    global last_folder
    if not TKINTER_AVAILABLE:
        print("❌ 檔案對話框不可用，請手動輸入檔案路徑")
        return
    
    try:
        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)
        
        file_path = filedialog.askopenfilename(
            title="選擇 Excel 檔案",
            filetypes=[(
                "Excel 檔案", "*.xlsx *.xls"),
                ("所有檔案", "*.*")
            ],
            initialdir=last_folder
        )
        
        root.destroy()
        
        if file_path:
            excel_file_input.value = file_path
            last_folder = os.path.dirname(file_path)
            print(f"✓ 已選擇: {os.path.basename(file_path)}")
            
    except Exception as e:
        print(f"❌ 選擇檔案時發生錯誤: {str(e)}")

excel_file_input = widgets.Text(
    value='',
    placeholder='選擇 Excel 檔案...',
    description='Excel 檔案:',
    layout=widgets.Layout(width='450px')
)

browse_button = widgets.Button(
    description='瀏覽...',
    layout=widgets.Layout(width='100px'),
    disabled=not TKINTER_AVAILABLE
)

browse_button.on_click(lambda x: select_excel_file())

reviewer_column_input = widgets.Text(
    value='Reviewer',
    description='審查者欄位:',
    layout=widgets.Layout(width='300px')
)

display(widgets.HBox([excel_file_input, browse_button]))
display(reviewer_column_input)

# === 選項設定 ===
display(HTML("<h3>⚙️ 步驟 4: 選項設定</h3>"))

copy_word_check = widgets.Checkbox(
    value=True,
    description='複製 Word 文件 (.doc, .docx)'
)

copy_pdf_check = widgets.Checkbox(
    value=True,
    description='複製 PDF 文件 (.pdf)'
)

upload_to_sharepoint_check = widgets.Checkbox(
    value=False,
    description='上傳檔案到 SharePoint（測試功能）'
)

# v2.0 新增：處理方法選擇
processing_method = widgets.RadioButtons(
    options=[
        ('刪除列（保留資料驗證）', 'delete_rows'),
        ('複製檔案後處理', 'copy_file_first'),
        ('Excel 自動化（僅 Windows）', 'win32com'),
        ('隱藏列（原始方法）', 'hide_rows')
    ],
    value='delete_rows',
    description='處理方法:',
    disabled=False
)

display(copy_word_check)
display(copy_pdf_check)
display(upload_to_sharepoint_check)
display(HTML("<br><b>Excel 處理方法（v2.0）:</b>"))
display(processing_method)

# === 處理按鈕 ===
process_button = widgets.Button(
    description='處理 Excel 檔案',
    layout=widgets.Layout(width='200px', height='40px')
)

# === 輸出區域 ===
output = widgets.Output()
sharing_container = widgets.VBox()

display(HTML("<br>"))
display(process_button)
display(output)
display(sharing_container)

print("\n✅ 介面已就緒！")

## 步驟 8: 審查者選擇介面

In [ ]:
def create_reviewer_sharing_ui(reviewers: List[str]):
    """建立審查者選擇與分享介面"""
    global reviewer_data
    
    # 初始化審查者資料
    reviewer_data = {}
    
    # 建立 UI 元件
    select_all = widgets.Checkbox(
        value=True,
        description='全選',
        indent=False,
        layout=widgets.Layout(width='100px')
    )
    
    reviewer_widgets = []
    
    # 查找所有審查者的 email
    with output:
        print("\n🔍 正在查找審查者 email...")
    
    for i, reviewer in enumerate(reviewers):
        reviewer_str = str(reviewer).strip()
        
        # 建立勾選框
        checkbox = widgets.Checkbox(
            value=True,
            description='',
            indent=False,
            layout=widgets.Layout(width='30px')
        )
        
        # 建立名稱標籤
        name_label = widgets.Label(
            value=reviewer_str[:20] + '...' if len(reviewer_str) > 20 else reviewer_str,
            layout=widgets.Layout(width='150px')
        )
        
        # 查找 email
        email = lookup_user_email(reviewer_str) if access_token else None
        
        # 建立 email 輸入框
        email_input = widgets.Text(
            value=email or '',
            placeholder='輸入 email 地址',
            layout=widgets.Layout(width='250px')
        )
        
        # 建立狀態標籤
        if email:
            status_html = "<span style='color:green'>✓ 已找到</span>"
        else:
            status_html = "<span style='color:orange'>⚠ 需手動輸入</span>"
        
        status_label = widgets.HTML(
            value=status_html,
            layout=widgets.Layout(width='120px')
        )
        
        # 儲存資料
        reviewer_data[reviewer_str] = {
            'checkbox': checkbox,
            'email_input': email_input,
            'status_label': status_label,
            'selected': True,
            'email': email,
            'status': 'ready'
        }
        
        # 建立列
        row = widgets.HBox([
            checkbox,
            name_label,
            email_input,
            status_label
        ])
        
        reviewer_widgets.append(row)
        
        # 顯示進度
        if (i + 1) % 5 == 0 or i == len(reviewers) - 1:
            with output:
                print(f"  已處理 {i + 1}/{len(reviewers)} 位審查者")
    
    # 全選處理函數
    def on_select_all_change(change):
        for data in reviewer_data.values():
            data['checkbox'].value = change['new']
    
    select_all.observe(on_select_all_change, names='value')
    
    # 分享按鈕
    share_button = widgets.Button(
        description='分享給選定的審查者',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # 進度條
    progress = widgets.IntProgress(
        value=0,
        min=0,
        max=len(reviewers),
        description='進度:',
        layout=widgets.Layout(width='400px')
    )
    
    progress_label = widgets.Label(value='')
    
    # 狀態輸出
    status_output = widgets.Output()
    
    # 分享按鈕處理函數
    def on_share_click(b):
        with status_output:
            clear_output()
            share_folders_to_reviewers(progress, progress_label)
    
    share_button.on_click(on_share_click)
    
    # 建立 UI
    sharing_ui = widgets.VBox([
        widgets.HTML("<h3>📤 步驟 5: SharePoint 分享</h3>"),
        widgets.HTML("<p>選擇要分享資料夾的審查者：</p>"),
        widgets.HBox([select_all]),
        widgets.HTML("<hr>"),
        widgets.VBox(reviewer_widgets),
        widgets.HTML("<hr>"),
        widgets.HBox([share_button]),
        widgets.HBox([progress, progress_label]),
        status_output
    ])
    
    return sharing_ui

def share_folders_to_reviewers(progress_widget, progress_label):
    """批次分享資料夾給審查者"""
    global site_id
    
    if not site_id:
        print("❌ 請先驗證 SharePoint 網站")
        return
    
    # 取得選定的審查者
    selected_reviewers = []
    for reviewer, data in reviewer_data.items():
        if data['checkbox'].value:
            email = data['email_input'].value.strip()
            if email and validate_email(email):
                selected_reviewers.append((reviewer, email))
            else:
                print(f"⚠️ 跳過 {reviewer} - 無效的 email 地址")
    
    if not selected_reviewers:
        print("❌ 沒有選擇審查者或沒有提供 email 地址")
        return
    
    print(f"\n📤 開始分享給 {len(selected_reviewers)} 位審查者...\n")
    
    # 重設進度
    progress_widget.value = 0
    progress_widget.max = len(selected_reviewers)
    
    success_count = 0
    
    for i, (reviewer, email) in enumerate(selected_reviewers):
        progress_label.value = f"{reviewer}"
        
        # 更新狀態
        data = reviewer_data[reviewer]
        data['status_label'].value = "<span style='color:blue'>⏳ 分享中...</span>"
        
        try:
            # 分享資料夾
            folder_path = sanitize_folder_name(reviewer)
            success = share_folder_with_user(site_id, folder_path, email, 'write', notify=False)
            
            if success:
                data['status_label'].value = "<span style='color:green'>✓ 已分享</span>"
                print(f"✓ {reviewer}: 資料夾已分享給 {email}")
                success_count += 1
            else:
                data['status_label'].value = "<span style='color:red'>✗ 分享失敗</span>"
                print(f"✗ {reviewer}: 分享失敗")
                
        except Exception as e:
            data['status_label'].value = "<span style='color:red'>✗ 錯誤</span>"
            print(f"✗ {reviewer}: 錯誤 - {str(e)}")
        
        # 更新進度
        progress_widget.value = i + 1
        time.sleep(0.5)  # 避免速率限制
    
    # 完成總結
    progress_label.value = "完成！"
    print(f"\n✅ 分享完成！成功分享給 {success_count}/{len(selected_reviewers)} 位審查者。")
    
    if site_url:
        print(f"\n📁 SharePoint 網站: {site_url}")
        print("審查者可以在 SharePoint 上找到他們的專屬資料夾。")

print("✓ 審查者選擇介面已就緒")

## 步驟 9: 主要處理函數

In [ ]:
def process_excel_file(button):
    """主要處理函數"""
    with output:
        clear_output()
        
        # 驗證輸入
        if not excel_file_input.value:
            print("❌ 請選擇 Excel 檔案")
            return
        
        if not access_token:
            print("❌ 請先完成登入")
            return
        
        file_path = excel_file_input.value.strip()
        column_name = reviewer_column_input.value.strip()
        method = processing_method.value
        
        if not os.path.exists(file_path):
            print(f"❌ 找不到檔案: {file_path}")
            return
        
        # 檢查處理方法
        if method == 'win32com' and not WIN32COM_AVAILABLE:
            print("⚠️ Excel 自動化不可用，切換到刪除列方法")
            method = 'delete_rows'
        
        print(f"📁 處理中: {os.path.basename(file_path)}")
        print(f"📊 審查者欄位: {column_name}")
        print(f"🔧 處理方法: {method}")
        print("=" * 50)
        
        try:
            # 讀取 Excel 檔案
            df = pd.read_excel(file_path, engine='openpyxl')
            
            if column_name not in df.columns:
                print(f"❌ 找不到欄位 '{column_name}'")
                print(f"可用欄位: {', '.join(df.columns)}")
                return
            
            # 取得唯一審查者
            reviewers = df[column_name].dropna().unique().tolist()
            print(f"✓ 找到 {len(reviewers)} 位審查者")
            
            # 處理每位審查者
            base_dir = os.path.dirname(file_path)
            processed = 0
            
            for reviewer in reviewers:
                print(f"\n📝 處理中: {reviewer}")
                
                success, folder_path, filename = process_reviewer_excel(
                    file_path, reviewer, column_name, base_dir, processing_method=method
                )
                
                if success:
                    print(f"  ✓ 已建立: {filename}")
                    
                    # 複製相關文件
                    if copy_word_check.value or copy_pdf_check.value:
                        copied = copy_selected_documents(
                            base_dir, folder_path,
                            copy_word=copy_word_check.value,
                            copy_pdf=copy_pdf_check.value
                        )
                        if copied:
                            print(f"  ✓ 已複製 {len(copied)} 個文件")
                    
                    processed += 1
            
            # 總結
            print("\n" + "=" * 50)
            print(f"✅ 處理完成！")
            print(f"📊 已處理 {processed}/{len(reviewers)} 位審查者")
            print(f"📁 輸出位置: {base_dir}")
            
            if method == 'delete_rows':
                print("\n✅ 使用刪除列方法，資料驗證應該已保留")
            elif method == 'hide_rows':
                print("\n⚠️ 使用隱藏列方法，資料驗證可能會遺失")
            
            # 建立分享介面
            global sharing_container
            sharing_ui = create_reviewer_sharing_ui(reviewers)
            sharing_container.children = [sharing_ui]
            
            print("\n💡 提示：")
            print("1. 檢查並修改審查者的 email 地址")
            print("2. 選擇要分享的審查者")
            print("3. 點擊『分享給選定的審查者』按鈕")
            
        except Exception as e:
            print(f"\n❌ 發生錯誤: {str(e)}")
            import traceback
            traceback.print_exc()

# 連接處理函數到按鈕
process_button.on_click(process_excel_file)
print("✓ 主要處理函數已就緒")

## 使用說明

### 設定步驟

1. **Azure AD 應用程式設定**
   - 在 Azure Portal 中註冊應用程式
   - 授予必要的 API 權限：
     - User.Read
     - User.ReadBasic.All
     - Sites.ReadWrite.All
     - Files.ReadWrite.All
   - 設定重新導向 URI: `http://localhost:8400`
   - 複製 Client ID 和 Tenant ID

2. **環境變數設定（選擇性）**
   ```bash
   # 建立 .env 檔案
   AZURE_CLIENT_ID=your-client-id
   AZURE_TENANT_ID=your-tenant-id
   ```

### 使用流程

1. **認證**
   - 輸入 Azure AD 憑證（或從環境變數自動載入）
   - 點擊『登入』按鈕
   - 在瀏覽器中完成 SSO 登入

2. **設定 SharePoint**
   - 輸入 SharePoint 網站 URL
   - 點擊『驗證網站』確認連線

3. **處理 Excel**
   - 選擇要處理的 Excel 檔案
   - 確認審查者欄位名稱
   - 選擇要複製的文件類型
   - **選擇處理方法**（v2.0 新增）
   - 點擊『處理 Excel 檔案』

4. **分享資料夾**
   - 系統會自動查找審查者的 email
   - 您可以手動修改或輸入 email
   - 選擇要分享的審查者
   - 點擊『分享給選定的審查者』

### v2.0 新功能：處理方法說明

1. **刪除列（保留資料驗證）**
   - 最推薦的方法
   - 刪除不相關的列，保留資料驗證
   - 適用於大部分情況

2. **複製檔案後處理**
   - 先複製整個檔案，再進行處理
   - 較安全但較慢

3. **Excel 自動化（僅 Windows）**
   - 使用 Windows COM 自動化
   - 最可靠但僅限 Windows

4. **隱藏列（原始方法）**
   - 原始方法，可能會遺失資料驗證
   - 不建議使用

### 功能特色

- ✅ **SSO 整合**：使用現有的企業登入
- ✅ **自動 Email 查找**：從 M365 目錄自動查找
- ✅ **批次處理**：一次處理多位審查者
- ✅ **即時進度**：顯示處理和分享進度
- ✅ **錯誤處理**：自動重試和錯誤提示
- ✅ **資料驗證保留**：v2.0 新增多種處理方法

### 疑難排解

1. **登入失敗**
   - 確認 Client ID 和 Tenant ID 正確
   - 確認應用程式有正確的權限

2. **找不到 Email**
   - 手動輸入正確的 email 地址
   - 確認審查者名稱與 M365 目錄相符

3. **分享失敗**
   - 確認您有該 SharePoint 網站的權限
   - 確認網站 URL 正確
   - 檢查網路連線

4. **速率限制**
   - 系統會自動處理速率限制
   - 大量分享時請耐心等待

5. **資料驗證遺失**
   - 使用『刪除列』方法而非『隱藏列』
   - 確認原始 Excel 檔案的資料驗證設定正確
   - 如果仍有問題，嘗試使用 Windows Excel 自動化方法